<h1><u>PROJECT</u> :</h1>
<h1>Extracting and Visualizing Stock Data</h1><BR>
<h2>Description</h2>


Extracting essential data from a dataset and displaying it; so that correct decisions can be made based on the data. In this project some essential data for a few specific stocks extracted and is then displayed in interactive graphs at the end of the notebook.


In [104]:
# !pip install yfinance
# !pip install pandas
# !pip install requests
# !pip install bs4
# !pip install plotly

In [105]:
import yfinance as yf
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

## Graphing Function:


In [106]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    
    stock_data_specific = stock_data[stock_data.Date <= datetime.today()]
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-06-30']
    
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), 
                             y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), 
                             y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False, height=900, title=stock, xaxis_rangeslider_visible=True)
    
    fig.show()
    print(type(stock_data.Date))

## Taking Inputs For Desired Stocks

In [107]:
n = int(input("Enter number of stocks for comparision : "))
tickers = []
stockNames = []
for i in range(0,n):
    print("Enter name of stock ",i+1," :", end="")
    temp1 = input()
    print("Enter its Ticker Symbol :", end="")
    temp2 = input()
    stockNames.append(temp1)
    tickers.append(temp2)

Enter number of stocks for comparision : 1
Enter name of stock  1  :upwork
Enter its Ticker Symbol :upwk


<B>Note:-</B>  Always make sure to convert all the ticker symbols to upper strings

In [108]:
for i in range(n):
    tickers[i] = tickers[i].upper()
print(stockNames)
print(tickers)

['upwork']
['UPWK']


## Using yfinance to Extract Stock Prices


Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is `TSLA`.


Using the ticker object and the function `history` extract stock information and saving it in a dataframe named `tesla_data`. S `period` parameter is `max` so we get information for the maximum amount of time.



**Reset the index** using the `reset_index(inplace=True)` function on the tesla_data DataFrame and display the first five rows of the `tesla_data` dataframe using the `head` function.

In [109]:
stock_data = {}
for i in range(n):
    stock = yf.Ticker(tickers[i])
    sd = stock.history(period='max')
    stock_data[i] = sd
    stock_data[i].reset_index(inplace=True)


In [110]:
for key in stock_data.keys():
    print(stockNames[key])
    print(stock_data[key].tail(5))
    print(" "*30)
    print("-~-"*30)
    print(" "*30)

upwork
          Date       Open       High        Low      Close   Volume  \
743 2021-09-16  46.930000  47.939999  46.529999  47.509998   852400   
744 2021-09-17  47.770000  50.080002  47.759998  49.700001  3190400   
745 2021-09-20  46.919998  48.279999  46.860001  48.000000  1591900   
746 2021-09-21  48.549999  50.349998  48.049999  50.020000  1361400   
747 2021-09-22  50.270000  51.750000  50.200001  51.580002   978700   

     Dividends  Stock Splits  
743          0             0  
744          0             0  
745          0             0  
746          0             0  
747          0             0  
                              
-~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~-
                              


## Using Webscraping to Extract Tesla Revenue Data


Revenues of stocks is extracted from the website [https://www.macrotrends.net/stocks]

Parse the html data using `beautiful_soup`.


Using `BeautifulSoup` or the `read_html` function, the table with `Quarterly Revenue` is extracted and stored into a dataframe named `stock_revenue`. The dataframe should have columns `Date` and `Revenue`.


 Remove the comma and dollar sign & null or empty strings from the `Revenue` column.


In [111]:
stock_revenue = {}

In [112]:
for i in range(n):
    url = "https://www.macrotrends.net/stocks/charts/" + tickers[i] + "/" + stockNames[i] + "/revenue"
    html_data = requests.get(url).text
    soup = BeautifulSoup(html_data, "html5lib")
    stock_revenue_temp = pd.DataFrame(columns=["Date","Revenue"])
    for row in soup.find_all("tbody")[1].find_all("tr"):
        col = row.find_all("td")
        date = col[0].text
        revenue = col[1].text

        stock_revenue_temp = stock_revenue_temp.append({"Date":date, "Revenue":revenue},ignore_index=True)
        
    stock_revenue_temp["Revenue"] = stock_revenue_temp['Revenue'].str.replace(',|\$',"")
    stock_revenue_temp.dropna(inplace=True)
    stock_revenue_temp = stock_revenue_temp[stock_revenue_temp['Revenue'] != ""]
    
    stock_revenue[i] = stock_revenue_temp

C:\Users\wadhw\AppData\Local\Temp/ipykernel_944/956665426.py:13: FutureWarning:

The default value of regex will change from True to False in a future version.



The top 5 rows of the `stock_revenue[i]` dataframe

In [113]:
for key in stock_revenue.keys():
    print(stockNames[key])
    print(stock_revenue[key].tail(5))
    print(" "*30)
    print("-~-"*30)
    print(" "*30)

upwork
          Date Revenue
10  2018-12-31      67
11  2018-09-30      64
12  2018-06-30      63
13  2018-03-31      59
15  2017-09-30      52
                              
-~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~--~-
                              


## Plotting the interactive visual Stock Graphs

Using the `make_graph` function to graph the Stock Data of all the stocks data with the title as the name of the company


In [114]:
for i in range(n):
    make_graph(stock_data[0], stock_revenue[i], stockNames[i])

<class 'pandas.core.series.Series'>
